In [ ]:
"""
!pip install -q sciunit
!pip install -q neo
!pip install -q elephant
!pip install -q gitpython
!pip install -q efel
"""

In [ ]:
!pip install scipy seaborn
!pip install -U git+https://github.com/appukuttan-shailesh/hbp-validation-client.git
!pip install --quiet fpdf PyPDF2 tabulate
!pip install ./bbp-services-0.0.15/ ./task-types-0.0.10/ hbp_service_client

In [1]:
import models
model2Test = models.CA1_laminar_distribution_synapses() 
from hbp_validation_framework import utils
utils.run_test(environment="production", hbp_username="pedroernesto", model=model2Test, test_alias="CA1-LDsyn-FT1-gof-Test", test_version="1.0")


/home/pedroernesto/miniconda2/lib/python2.7/site-packages/pandas/core/nanops.py:39: UserWarning: The installed version of bottleneck 0.8.0 is not supported in pandas and will be not be used
The minimum supported version is 1.0.0

  ver=ver, min_ver=_MIN_BOTTLENECK_VERSION), UserWarning)


----------------------------------------------
('Model name: ', 'CA1_laminar_distribution_synapses')
('Model type: ', <class 'models.CA1_laminar_distribution_synapses'>)
----------------------------------------------
----------------------------------------------
('Test name: ', 'CA1 laminar_distribution_synapses Test')
('Test type: ', <class 'HippoNetworkUnit.tests.test_CA1_laminar_distribution_synapses_FreemanTukey1.CA1_laminar_distribution_synapses_FreemanTukey1Test'>)
----------------------------------------------
       FreemanTukey1-score   p-value
AA                1.352743  0.098159
BP                1.075224  0.915977
BS                1.100946  0.926553
CCKBC             0.260937  0.514982
Ivy               1.198244  0.961850
OLM               1.277291  0.106840
PC                0.355081  0.286855
PPA               1.414214  0.091578
SCA               1.111951  0.930936
Tri               0.160865  0.471068 

----------------------------------------------
('Score: ', 0.355080

u'f39b3019-3a56-4eb8-b566-a30102d97309'